In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

Duplicate key in file WindowsPath('d:/source/Anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family:  Microsoft JhengHei')


In [3]:
# prediction_df= pd.read_csv("datas/model_predictions.csv")
model_df= pd.read_csv("datas/predictions.csv")


In [4]:
model_df.columns

Index(['datetime', '成交口數', '成交筆數', '主動買入筆數', '主動賣出筆數', '距離開盤時間分鐘數', 'open',
       'high', 'low', 'close', 'volume', '高低點幅', '收漲跌幅', '近10分鐘的收漲跌幅',
       '近10分鐘高低點幅', '近10分鐘高點到現在的漲跌幅', '近10分鐘低點到現在的漲跌幅', '前10分鐘成交量總和',
       '近30分鐘的收漲跌幅', '近30分鐘高低點幅', '近30分鐘高點到現在的漲跌幅', '近30分鐘低點到現在的漲跌幅',
       '前30分鐘成交量總和', '近60分鐘的收漲跌幅', '近60分鐘高低點幅', '近60分鐘高點到現在的漲跌幅',
       '近60分鐘低點到現在的漲跌幅', '前60分鐘成交量總和', '近120分鐘的收漲跌幅', '近120分鐘高低點幅',
       '近120分鐘高點到現在的漲跌幅', '近120分鐘低點到現在的漲跌幅', '前120分鐘成交量總和', '台灣-美國高低掛', '星期幾',
       'vwap', 'bollinger_upper', 'bollinger_lower', 'bollinger_bandwidth',
       'rsi', 'macd_line', 'signal_line', '%K', '%D', 'atr',
       'predicted_value'],
      dtype='object')

In [5]:
model_df.index

RangeIndex(start=0, stop=3000, step=1)

In [6]:
strategy_df = model_df[["datetime","close", "predicted_value"]].copy()

In [7]:
#統計model_df中,Predicted的分佈情況為何，並計算標準差
strategy_df.describe()


,close,predicted_value
count,3000.000000,3000.000000
mean,42754.492667,12.611830
std,371.595632,6.259276
min,42130.000000,4.414376
25%,42478.000000,6.932455
50%,42630.500000,12.093729
75%,43051.000000,16.302498
max,43569.000000,41.020568


In [8]:
# 將model_df中,"10分鐘後的漲跌幅_預測"的值轉換為二值化，若"10分鐘後的漲跌幅_預測"大於等於mean+std則為1，若"10分鐘後的漲跌幅_預測"小於等於mean-std則為-1，其餘為0
strategy_df["預測5分鐘後漲跌"] = np.where(
    strategy_df["predicted_value"] >= strategy_df["predicted_value"].quantile(0.75),
    1,
    np.where(
        strategy_df["predicted_value"] <= strategy_df["predicted_value"].quantile(0.25),
        -1,
        0,
    ),
)

In [11]:
要統計_df=strategy_df
# 創建計數變量
pred_1_count = 0
pred_1_success = 0
pred_neg1_count = 0
pred_neg1_success = 0

# 遍歷數據以滿足條件
for i in range(len(要統計_df) - 2):  # 確保能夠訪問i+2行
    current_close = 要統計_df.iloc[i]["close"]
    future_close = 要統計_df.iloc[i + 2]["close"]
    prediction = 要統計_df.iloc[i]["預測5分鐘後漲跌"]

    # 當預測為1時
    if prediction == 1:
        pred_1_count += 1
        if future_close > current_close+5:
            pred_1_success += 1

    # 當預測為-1時
    elif prediction == -1:
        pred_neg1_count += 1
        if future_close < current_close-5:
            pred_neg1_success += 1

# 計算正確比例 / or 猜錯比例
pred_1_success_rate = pred_1_success / pred_1_count * 100 if pred_1_count > 0 else None
pred_neg1_success_rate = (
    pred_neg1_success / pred_neg1_count * 100 if pred_neg1_count > 0 else None
)

pred_1_success_rate, pred_neg1_success_rate

(48.93333333333334, 29.733333333333334)

In [34]:
要繪製_df = strategy_df.copy()

In [35]:
要繪製_df

,datetime,close,predicted_value,預測10分鐘後漲跌
0,2024-10-02 02:35:00,42627.0,1.047947,1
1,2024-10-02 02:40:00,42633.0,0.134251,1
2,2024-10-02 02:45:00,42622.0,0.030630,1
3,2024-10-02 02:50:00,42609.0,-0.952274,0
4,2024-10-02 02:55:00,42612.0,-1.120601,-1
...,...,...,...,...
2995,2024-10-29 03:35:00,42652.0,-1.332978,-1
2996,2024-10-29 03:40:00,42654.0,-1.261483,-1
2997,2024-10-29 03:45:00,42640.0,-1.280301,-1
2998,2024-10-29 03:50:00,42624.0,-0.995497,0


In [37]:
# %matplotlib inline
%matplotlib qt

def plot_price_predictions(要繪製_df):
    # 創建圖表
    plt.figure(figsize=(15, 7))

    # 繪製收盤價折線圖
    plt.plot(
        要繪製_df['datetime'],
        要繪製_df["close"],
        color="blue",
        linewidth=1,
        label="收盤價",
    )

    # 設置箭號樣式
    arrow_params = dict(arrowstyle="^", linestyle="-")


    # 預測漲跌的箭號（深色）
    for idx, row in 要繪製_df.iterrows():
        if row["預測10分鐘後漲跌"] == 1:
            plt.plot(
                row["datetime"],
                row["close"],
                marker="^",
                color="red",
                markersize=10,
                label="預測上漲" if idx == 要繪製_df.index[0] else "",
            )
        elif row["預測10分鐘後漲跌"] == -1:
            plt.plot(
                row["datetime"],
                row["close"],
                marker="v",
                color="green",
                markersize=10,
                label="預測下跌" if idx == 要繪製_df.index[0] else "",
            )

    # 設置圖表樣式
    plt.title("價格走勢與預測", fontsize=12)
    plt.xlabel("時間")
    plt.ylabel("價格")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.legend(loc="best")

    # 調整x軸標籤角度，避免重疊
    plt.xticks(rotation=45)

    # 自動調整布局
    plt.tight_layout()

    return plt


# 使用示例：
plot_price_predictions(要繪製_df)
plt.show()